In [13]:
%load_ext autoreload
%load_ext my_magics
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The my_magics extension is already loaded. To reload it, use:
  %reload_ext my_magics


In [14]:
# %%isortify
import json

import config
import matplotlib.pyplot as plt
import torch
import utils
from IPython.display import HTML, display
from matplotlib import animation
from torch.utils.data import DataLoader
from torchvision.transforms import v2

from data_augmentation import VideoTransform
from video_dataset import VideoDataset

# TODO: test shuffle, kfold

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Load the transforms, datasets and dataloaders for each of the databases.

In [4]:
transform_hockey_fights = VideoTransform(dataset="HockeyFights", json_file="test.json")
dataset_hockey_fights = VideoDataset(
    dataset="HockeyFights", transformations=transform_hockey_fights
)
dataloader_hockey_fights = DataLoader(
    dataset=dataset_hockey_fights, batch_size=1, collate_fn=utils.collate_fn_pad
)

In [5]:
transform_rwf = VideoTransform(dataset="RWF-2000", json_file="test.json")
dataset_rwf = VideoDataset(dataset="RWF-2000", transformations=transform_rwf)
dataloader_rwf = DataLoader(
    dataset=dataset_rwf, batch_size=1, collate_fn=utils.collate_fn_pad
)

In [6]:
transform_fight_surv = VideoTransform(
    dataset="Fight-Surveillance", json_file="test.json"
)
dataset_fight_surv = VideoDataset(
    dataset="Fight-Surveillance", transformations=transform_rwf
)
dataloader_fight_surv = DataLoader(
    dataset=dataset_fight_surv, batch_size=1, collate_fn=utils.collate_fn_pad
)

Testing the VideoDataset and VideoTransform classes:
<br>
1. Test the videos and their label to match.

In [21]:
for dataset, dataloader in [
    [dataset_hockey_fights, dataloader_hockey_fights],
    [dataset_fight_surv, dataloader_fight_surv],
    [dataset_rwf, dataloader_rwf],
]:
    print("Dataset train:", dataset.dataset)

    for i, batch in enumerate(dataloader):
        video, label, _ = batch
        print(type(video))
        if label:
            assert dataset._fight_folder_path.joinpath(dataset._train[i][0]).exists()
        else:

            assert dataset._non_fight_folder_path.joinpath(
                dataset._train[i][0]
            ).exists()

    dataset.flag = True
    print("Dataset val:", dataset.dataset)

    for i, batch in enumerate(dataloader):
        video, label, _ = batch
        if label:
            assert dataset._fight_folder_path.joinpath(dataset._val[i][0]).exists()
        else:
            assert dataset._non_fight_folder_path.joinpath(dataset._val[i][0]).exists()

Dataset train: HockeyFights


TypeError: 'Image' object is not iterable

2. Play the videos to check the rendering of the data together with the transforms.

In [7]:
animations_html = []

#! removed from code normalization to visualize the data

# give 3 examples for each dataset
for dataloader in [dataloader_rwf, dataloader_hockey_fights, dataloader_fight_surv]:

    for i, batch in enumerate(dataloader):
        fig, ax = plt.subplots(figsize=(3, 3))
        frames = batch[0][0].permute(0, 2, 3, 1)

        # define an animation to play th video
        def animate(frame):
            ax.clear()
            ax.imshow(frame.float().clamp(0, 1))
            ax.axis("off")

        ani = animation.FuncAnimation(fig, animate, frames=frames, interval=30)
        animations_html.append(ani.to_jshtml())
        plt.close(fig)

        break

# create columns for output trough html
html_output = '<div style="display: flex; flex-wrap: wrap;">'
for animation_html in animations_html:
    html_output += f'<div style="width: 30%;">{animation_html}</div>'
html_output += "</div>"

display(HTML(html_output))

TypeError: 'Image' object is not iterable

3. Test loading and dumping the augmentation values form VideoTransform.

In [9]:
with open("test.json", mode="r") as file:
    aug_val = json.load(file)["RWF-2000"]

# modified test.json with another value (360, 360) for testing
aug_val["resize"] = [255, 255]
transform_rwf.save(aug_val, dump=True)
aug_val_class = transform_rwf.json(load=True)
assert aug_val_class == aug_val

aug_val["resize"] = [3, 3]
transform_rwf.save(aug_val, dump=False)
aug_val_class = transform_rwf.json()
assert aug_val_class == aug_val

aug_val_class = transform_rwf.json(load=True)
assert aug_val_class != aug_val

try:
    aug_val["new"] = "identity"
    transform_rwf.save(aug_val)
except ValueError as e:
    assert str(e) == "Dictionary doesn't match the required structure."

print("Assertion passed")

Assertion passed


4. Check the normalization of data.

In [ ]:
torch.cuda.empty_cache()

# same transforms from VideoDataset but with normalization applied
transform_rwf = v2.Compose(
    [
        config.BASIC_TRANSFORMS,
        v2.Normalize(mean=[0.3910, 0.3840, 0.3789], std=[0.2617, 0.2609, 0.2615]),
    ]
)
transform_hockey_fights = v2.Compose(
    [
        config.BASIC_TRANSFORMS,
        v2.Normalize(mean=[0.3688, 0.3700, 0.3593], std=[0.3657, 0.3694, 0.3645]),
    ]
)
transform_another_dataset = v2.Compose(
    [
        config.BASIC_TRANSFORMS,
        v2.Normalize(mean=[0.4278, 0.4154, 0.4131], std=[0.2538, 0.2515, 0.2538]),
    ]
)

dataset_rwf = VideoDataset(dataset="RWF-2000", transformations=transform_rwf)
dataset_hockey_fights = VideoDataset(
    dataset="HockeyFights", transformations=transform_hockey_fights
)
dataset_fight_surv = VideoDataset(
    dataset="Fight-Surveillance", transformations=transform_fight_surv
)

for dataset in [dataset_rwf, dataset_hockey_fights, dataset_fight_surv]:
    mean, std = utils.compute_mean_std(dataset, device)

    is_mean_zero = torch.allclose(mean, torch.zeros_like(mean), atol=0.01)
    is_std_one = torch.allclose(std, torch.ones_like(std), atol=0.01)

    assert is_mean_zero and is_std_one

    print(f"Assert for {dataset.dataset} passed")

Computation of mean and std for each of the datasets.

In [ ]:
torch.cuda.empty_cache()

dataset_hockey_fights = VideoDataset(dataset="HockeyFights")
dataset_rwf = VideoDataset(dataset="RWF-2000")
dataset_fight_surv = VideoDataset(dataset="Fight-Surveillance")

mean, std = utils.compute_mean_std(dataset_hockey_fights, device)
print(mean, std)
mean, std = utils.compute_mean_std(dataset_rwf, device)
print(mean, std)
mean, std = utils.compute_mean_std(dataset_fight_surv, device)
print(mean, std)

# hf
# tensor([0.3688, 0.3700, 0.3593], device='cuda:0') tensor([0.3657, 0.3694, 0.3645], device='cuda:0')
# rwf
# tensor([0.3910, 0.3840, 0.3789], device='cuda:0') tensor([0.2617, 0.2609, 0.2615], device='cuda:0')
# fs
# tensor([0.4278, 0.4154, 0.4131], device='cuda:0') tensor([0.2538, 0.2515, 0.2538], device='cuda:0')

# TODO: see what should be done for 3rd dataset for the variation of length to calc properly the mean and std